# Data for Learning

In [333]:
import pandas as pd # import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [336]:
%cd CSV_file_from_2007_to_2017

/Users/Seychelles/Desktop/GoogleTensorflow/Data_Collector/CSV_file_from_2007_to_2017


In [337]:
df_n = pd.read_csv("test.csv") #load data set saved from previous ops


In [338]:
df_n['PM10']= pd.to_numeric(df_n['PM10']) # Convert PM_10 column to Numeric Values

In [339]:
df_n['PM10'].replace('NaN', np.nan)[1:3] #replace NAn with zero np.nan

1      NaN
2    61.67
Name: PM10, dtype: float64

In [340]:
df_n[1:2]

,Unnamed: 0,date,Lat,Lon,PM10,NAME
1,1,2007-01-26 10:00:00,42.0036,21.4636,NaN,Gazi Baba


In [341]:
df_n.date= pd.to_datetime(df_n.date)  # CONVER TO DATE TIME 

In [342]:
df_n[0:2] # 

,Unnamed: 0,date,Lat,Lon,PM10,NAME
0,0,2007-01-26 10:00:00,41.9925,21.423611,NaN,Centar
1,1,2007-01-26 10:00:00,42.0036,21.463600,NaN,Gazi Baba


In [343]:
df_n.drop(['Lat','Lon','NAME','Unnamed: 0'], axis = 1, inplace = True, errors = 'ignore')
#drop non useful columns

In [344]:
df_n.date= pd.to_datetime(df_n.date)  # Convert to dat time

In [345]:
df_n.sort_values(by='date',ascending=True,inplace=True) # sort data sets

In [346]:
df_n[0:4]

,date,PM10
9414,2007-01-02,NaN
9408,2007-01-02,NaN
9411,2007-01-02,NaN
9412,2007-01-02,204.92


In [347]:
df_n = df_n.reset_index(drop=True) # reset index

In [348]:
df_n[0:5]

,date,PM10
0,2007-01-02,NaN
1,2007-01-02,NaN
2,2007-01-02,NaN
3,2007-01-02,204.92
4,2007-01-02,NaN


In [13]:
#df_n.set_index('date',inplace=True); # set index as date 
#del df_n.index.name  # delete index name

<p> Count the Null Values present</p>

In [349]:
df_n.isnull().any() # date has no null value while PM10 has null value

date    False
PM10     True
dtype: bool

In [350]:
df_n.isnull().sum() # null value counts

date         0
PM10    255713
dtype: int64

<p><b> Define Function to Make column with -1 for Nan and 1 for Numeric Values</b></p>
<p> Always we define a function to Normalize and Standardize data sets <p>

In [351]:

def Make_Indicator_Column_For_Nan(x):
    if (np.isnan(x)):
        val = -1
    else:
        val = 1
    return val

def Normalize(x):
    x = (x - x.mean())/(x.max()-x.min())
    return(x);

def first_difference(x):
    difference = x - x.shift(1);
    return (difference);

def Standardize(x):
    x = (x - x.mean())/x.std()
    return(x);

def recover(original_PandasSeries,differenced_PandasSeries):
    '''
    Using the diffreence data set , we need to recover the initial 
    set, this function does that:
    Input: Differenced pandas series
           Original Series 
    Output: Compute the original
    
    The aim is that when we make prediction with the differenced 
    data sets we should be able to convert to original set.
    '''
    i= 0;
    c = len(original_PandasSeries);
    recovered_set =[];
    #print(b)
    for k in range(0,c):
        if (k ==0):
            temp = a[k];
            recovered_set.append(temp);
        else:
            i=i+1;
            temp = original_PandasSeries[k-1]+ differenced_PandasSeries[i];
            recovered_set.append(temp);
    recovered_setArray=np.array(recovered_set);
    return (recovered_setArray)        


#d["recovered_set"] = recovered # add to data set
#Or use differece differenced_PandasSeries + differenced_PandasSeries.shift(1)

<p> Apply the function Above </p>

In [352]:
df_n['PM10_null_pointers']=df_n.PM10.apply(Make_Indicator_Column_For_Nan)

In [353]:
df_n[1:3]

,date,PM10,PM10_null_pointers
1,2007-01-02,NaN,-1
2,2007-01-02,NaN,-1


In [354]:
df_Normalized = df_n.copy(deep=True)  #Normalized

In [355]:
df_Standardized = df_n.copy(deep=True) # Standardized

In [356]:
df_Normalized.PM10 = Normalize(df_Normalized.PM10) #Normalize

In [357]:
df_Standardized.PM10 = Standardize(df_Standardized.PM10) #Standardize

In [358]:
df_Normalized.head()

,date,PM10,PM10_null_pointers
0,2007-01-02,NaN,-1
1,2007-01-02,NaN,-1
2,2007-01-02,NaN,-1
3,2007-01-02,0.118376,1
4,2007-01-02,NaN,-1


In [359]:
df_Standardized.head()

,date,PM10,PM10_null_pointers
0,2007-01-02,NaN,-1
1,2007-01-02,NaN,-1
2,2007-01-02,NaN,-1
3,2007-01-02,1.499629,1
4,2007-01-02,NaN,-1


<p> <b> Differenced Data      </b> </p>

In [365]:
df_differenced = df_Normalized.copy(deep=True)

In [366]:
df_differenced.fillna(0,inplace=True)   #fill na to compute first difference 

In [367]:
df_differenced.PM10 = first_difference(df_differenced.PM10) # first Difference

In [368]:
df_differenced.isnull().sum()

date                  0
PM10                  1
PM10_null_pointers    0
dtype: int64

In [369]:
df_differenced.fillna(0,inplace=True)   #fill na to compute first difference 

In [370]:
df_differenced[2340:2350]

,date,PM10,PM10_null_pointers
2340,2007-01-10 22:00:00,0.006117,-1
2341,2007-01-10 22:00:00,0.016462,1
2342,2007-01-10 22:00:00,-0.016462,-1
2343,2007-01-10 22:00:00,0.000000,-1
2344,2007-01-10 22:00:00,0.000000,-1
2345,2007-01-10 23:00:00,0.000000,-1
2346,2007-01-10 23:00:00,0.000000,-1
2347,2007-01-10 23:00:00,0.002336,1
2348,2007-01-10 23:00:00,-0.002336,-1
2349,2007-01-10 23:00:00,0.043752,1


# Import Additional feature sets

In [8]:
%ls


Cov_nonstationary.png               Var_nonstationary.png
EasternRegion_CO_airpointers.csv    WesternRegion_CO_airpointers.csv
EasternRegion_NO2_airpointers.csv   WesternRegion_NO2_airpointers.csv
EasternRegion_O3_airpointers.csv    WesternRegion_O3_airpointers.csv
EasternRegion_PM10_airpointers.csv  WesternRegion_PM10_airpointers.csv
EasternRegion_PM25_airpointers.csv  WesternRegion_PM25_airpointers.csv
EasternRegion_SO2_airpointers.csv   WesternRegion_SO2_airpointers.csv
ImpurityList_with_Indicator         data.xlsx
Mean_nonstationary.png              newfile.xlsx
SkopjeRegion_CO_airpointers.csv     outfile
SkopjeRegion_NO2_airpointers.csv    outfile1
SkopjeRegion_O3_airpointers.csv     processed_with_columns_droped.csv
SkopjeRegion_PM10_airpointers.csv   raw_processed.csv
SkopjeRegion_PM25_airpointers.csv   test.csv
SkopjeRegion_SO2_airpointers.csv


In [951]:
# Import Data sets

In [144]:
df_CO = pd.read_csv('SkopjeRegion_CO_airpointers.csv');
df_NO2 = pd.read_csv('SkopjeRegion_NO2_airpointers.csv');
df_O3 = pd.read_csv('SkopjeRegion_O3_airpointers.csv');
df_SO3 = pd.read_csv('SkopjeRegion_SO2_airpointers.csv');
df_PM25 = pd.read_csv('SkopjeRegion_PM25_airpointers.csv');

In [145]:
ImpurityList = [df_CO,df_NO2,df_O3,df_PM25] #create data set list

In [146]:
# Preprocess data set 

def preprocess(FrameList):
    '''
    Input:
         List of Dataframes
    Output:
         List of data frames with 4 dropped columns and renamed
    '''
    
    
    for i in range(0,len(FrameList)):
        print(i)
        column_name=FrameList[i].columns[0]
        column_replace = FrameList[i].columns[0][18:-5]
        print (column_replace)
        FrameList[i] = FrameList[i].rename(columns = {column_name:'x'});
        FrameList[i]=pd.DataFrame(FrameList[i].x.str.split(';',5).tolist(),columns = ['date','Lat','Lon',column_replace,'NAME'])
        FrameList[i].drop(['Lat','Lon','NAME'], axis = 1, inplace = True, errors = 'ignore')
    return (FrameList);
    

In [116]:
# Apply function on List
DataFrameList = preprocess(ImpurityList);

0
CO
1
NO2
2
O3
3
PM25


In [147]:
# Add Indicator for other variables for the impurity
for i in range(0,len(DataFrameList)):
    l= DataFrameList[i].columns[1]
    print(l)
    DataFrameList[i][str(l)]= pd.to_numeric(DataFrameList[i][str(l)]) #
    DataFrameList[i][str(l)].replace('NaN', np.NaN) #replace NAn with zero np.nan
    DataFrameList[i][str(l)+' indicators']= DataFrameList[i][str(l)].apply(Make_Indicator_Column_For_Nan)
    
    

CO
NO2
O3
PM25


In [117]:
DataFrameList[0][0:5]

,date,CO
0,26.01.2007 10,0.49
1,26.01.2007 10,0.78
2,26.01.2007 10,1.02
3,26.01.2007 10,0.91
4,26.01.2007 10,1.00


<p> Same data set with pickle</p>

In [17]:
import pickle as pick
with open('ImpurityList_with_Indicator', 'wb') as fp:
    pick.dump(DataFrameList, fp)

In [ ]:
import pickle as pick
with open ('ImpurityList_with_Indicator', 'rb') as fp:
    DataFrameList= pick.load(fp)

In [148]:
# Copy data set to define Normalized and Stabdardized sets
import copy
NewDataFrameListNormalized=copy.deepcopy(DataFrameList)
NewDataFrameListStandardized=copy.deepcopy(DataFrameList)

In [151]:
NewDataFrameListNormalized[0][0:8]

,date,CO,CO indicators
0,26.01.2007 10,0.49,1
1,26.01.2007 10,0.78,1
2,26.01.2007 10,1.02,1
3,26.01.2007 10,0.91,1
4,26.01.2007 10,1.00,1
5,26.01.2007 10,NaN,-1
6,26.01.2007 10,NaN,-1
7,15.01.2007 01,3.44,1


In [152]:
NewDataFrameListStandardized[0][0:8]

,date,CO,CO indicators
0,26.01.2007 10,0.49,1
1,26.01.2007 10,0.78,1
2,26.01.2007 10,1.02,1
3,26.01.2007 10,0.91,1
4,26.01.2007 10,1.00,1
5,26.01.2007 10,NaN,-1
6,26.01.2007 10,NaN,-1
7,15.01.2007 01,3.44,1


In [153]:
def datePreprocess(List):
    for i in range(0,len(List)):
        l = List[i].columns[0]
        print(l)
        #NewDataFrameListStandardized[i][str(l)] = Standardize(NewDataFrameListStandardized[i][str(l)]) #Normalize
        List[i][str(l)]= pd.to_datetime(List[i][str(l)])  # Convert to dat time
        List[i].sort_values(by=str(l),ascending=True,inplace=True) # sort data sets
        List[i] = List[i].reset_index(drop=True) # reset index
    return (List)  

In [154]:
NewDataFrameListNormalized_1 = datePreprocess(NewDataFrameListNormalized)

date
date
date
date


In [155]:
NewDataFrameListStandardized_2 = datePreprocess(NewDataFrameListStandardized)

date
date
date
date


In [156]:
NewDataFrameListStandardized_2[0][0:5]

,date,CO,CO indicators
0,2007-01-02,NaN,-1
1,2007-01-02,3.33,1
2,2007-01-02,4.13,1
3,2007-01-02,2.73,1
4,2007-01-02,NaN,-1


In [157]:
NewDataFrameListNormalized_1[0][0:5]

,date,CO,CO indicators
0,2007-01-02,NaN,-1
1,2007-01-02,3.33,1
2,2007-01-02,4.13,1
3,2007-01-02,2.73,1
4,2007-01-02,NaN,-1


In [167]:
id(NewDataFrameListNormalized)

4707627528

In [168]:
id(NewDataFrameListStandardized)

4707631688

In [161]:
import pickle as pick
with open('ImpurityList_Normalized', 'wb') as fp:
    pick.dump(NewDataFrameListNormalized_1, fp)

In [160]:
with open('ImpurityList_Standardized', 'wb') as fp:
    pick.dump(NewDataFrameListStandardized_2, fp)

In [162]:
NewDataFrameListNormalized_1[0]['CO'] =Normalize(NewDataFrameListNormalized[0]['CO'])
NewDataFrameListNormalized_1[1]['NO2']=Normalize(NewDataFrameListNormalized[1]['NO2'])
NewDataFrameListNormalized_1[2]['O3']=Normalize(NewDataFrameListNormalized[2]['O3'])
NewDataFrameListNormalized_1[3]['PM5']=Normalize(NewDataFrameListNormalized[3]['PM25'])

In [163]:
NewDataFrameListStandardized_2[0]['CO'] =Standardize(NewDataFrameListNormalized[0]['CO'])
NewDataFrameListStandardized_2[1]['NO2']=Standardize(NewDataFrameListNormalized[1]['NO2'])
NewDataFrameListStandardized_2[2]['O3']=Standardize(NewDataFrameListNormalized[2]['O3'])
NewDataFrameListStandardized_2[3]['PM25']=Standardize(NewDataFrameListNormalized[3]['PM25'])

In [164]:
NewDataFrameListNormalized_1[0]['CO'][0:5] #Normalize

0         NaN
1    0.044527
2    0.058631
3    0.033949
4         NaN
Name: CO, dtype: float64

In [166]:
NewDataFrameListStandardized_2[0]['CO'][0:5]

0         NaN
1    1.460885
2    1.923636
3    1.113822
4         NaN
Name: CO, dtype: float64

#These functions also does the Normalization. I used the syntax above for Clarificatio

for i in range(0,len(NewDataFrameListNormalized)):
        l = NewDataFrameListNormalized[i].columns[1];
        #print(l)
        NewDataFrameListNormalized[i][str(l)] = Normalize(NewDataFrameListNormalized[i][str(l)]); #Normalize

#These functions also does the Standardization. I used the syntax above for Clarificatio

for i in range(0,len(NewDataFrameListStandardized)):
        l = NewDataFrameListStandardized[i].columns[1];
        #print(l)
        NewDataFrameListStandardized[i][str(l)] = Standardize(NewDataFrameListStandardized[i][str(l)]); 
        #Standardize
  

In [214]:
NewDataFrameListStandardized_2[3][1:3]

,date,PM25,PM25 indicators
1,2007-01-02,NaN,-1
2,2007-01-02,NaN,-1


# Combining data sets


<p><b> Stabdardized Data set  </b></p>

In [256]:
df_Standardized.dtypes

date                  datetime64[ns]
PM10                         float64
PM10_null_pointers             int64
dtype: object

In [255]:
df_Standardized.head()

,date,PM10,PM10_null_pointers
0,2007-01-02,NaN,-1
1,2007-01-02,NaN,-1
2,2007-01-02,NaN,-1
3,2007-01-02,1.499629,1
4,2007-01-02,NaN,-1


In [257]:
df_Standardized['CO'] = NewDataFrameListStandardized_2[0]['CO']
df_Standardized['CO_null_pointers'] = NewDataFrameListStandardized_2[0]['CO indicators']

In [258]:
df_Standardized.head()

,date,PM10,PM10_null_pointers,CO,CO_null_pointers
0,2007-01-02,NaN,-1,NaN,-1
1,2007-01-02,NaN,-1,1.460885,1
2,2007-01-02,NaN,-1,1.923636,1
3,2007-01-02,1.499629,1,1.113822,1
4,2007-01-02,NaN,-1,NaN,-1


In [259]:
df_Standardized['NO2'] = NewDataFrameListStandardized_2[1]['NO2']
df_Standardized['NO2_null_pointers'] = NewDataFrameListStandardized_2[1]['NO2 indicators']

In [260]:
df_Standardized[1:2]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers
1,2007-01-02,NaN,-1,1.460885,1,NaN,-1


In [261]:
df_Standardized['O3'] = NewDataFrameListStandardized_2[2]['O3']
df_Standardized['O3_null_pointers'] = NewDataFrameListStandardized_2[2]['O3 indicators']

df_Standardized.drop(['O3'], axis = 1, inplace = True, errors = 'ignore')

In [262]:
df_Standardized[1:3]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers,O3,O3_null_pointers
1,2007-01-02,NaN,-1,1.460885,1,NaN,-1,NaN,-1
2,2007-01-02,NaN,-1,1.923636,1,NaN,-1,NaN,-1


In [263]:
df_Standardized['PM25'] = NewDataFrameListStandardized_2[3]['PM25']
df_Standardized['PM25_null_pointers'] = NewDataFrameListStandardized_2[3]['PM25 indicators']

In [264]:
df_Standardized[1:4]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers,O3,O3_null_pointers,PM25,PM25_null_pointers
1,2007-01-02,NaN,-1,1.460885,1,NaN,-1,NaN,-1,NaN,-1
2,2007-01-02,NaN,-1,1.923636,1,NaN,-1,NaN,-1,NaN,-1
3,2007-01-02,1.499629,1,1.113822,1,1.412657,1,-0.839846,1,NaN,-1


<p><b> Normalized Data set  </b></p>

In [265]:
df_Normalized[1:3]

,date,PM10,PM10_null_pointers
1,2007-01-02,NaN,-1
2,2007-01-02,NaN,-1


In [266]:
df_Normalized['CO'] = NewDataFrameListNormalized_1[0]['CO']
df_Normalized['CO_null_pointers'] = NewDataFrameListNormalized_1[0]['CO indicators']

In [267]:
df_Normalized[1:2]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers
1,2007-01-02,NaN,-1,0.044527,1


In [268]:
df_Normalized['NO2'] = NewDataFrameListNormalized_1[1]['NO2']
df_Normalized['NO2_null_pointers'] = NewDataFrameListNormalized_1[1]['NO2 indicators']

In [269]:
df_Normalized[1:2]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers
1,2007-01-02,NaN,-1,0.044527,1,NaN,-1


In [270]:
df_Normalized['O3'] = NewDataFrameListNormalized_1[2]['O3']
df_Normalized['O3_null_pointers'] = NewDataFrameListNormalized_1[2]['O3 indicators']

In [271]:
df_Normalized[1:3]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers,O3,O3_null_pointers
1,2007-01-02,NaN,-1,0.044527,1,NaN,-1,NaN,-1
2,2007-01-02,NaN,-1,0.058631,1,NaN,-1,NaN,-1


In [272]:
df_Normalized['PM25'] = NewDataFrameListNormalized_1[3]['PM25']
df_Normalized['PM25_null_pointers'] = NewDataFrameListNormalized_1[3]['PM25 indicators']

In [273]:
df_Normalized[1:4]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers,O3,O3_null_pointers,PM25,PM25_null_pointers
1,2007-01-02,NaN,-1,0.044527,1,NaN,-1,NaN,-1,NaN,-1
2,2007-01-02,NaN,-1,0.058631,1,NaN,-1,NaN,-1,NaN,-1
3,2007-01-02,0.118376,1,0.033949,1,0.013418,1,-0.081297,1,NaN,-1


<p><b> Differenced Data set  </b></p>

In [274]:
df_differenced_full = df_Normalized.copy(deep=True)

In [277]:
df_differenced_full[1:3]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers,O3,O3_null_pointers,PM25,PM25_null_pointers
1,2007-01-02,NaN,-1,0.044527,1,NaN,-1,NaN,-1,NaN,-1
2,2007-01-02,NaN,-1,0.058631,1,NaN,-1,NaN,-1,NaN,-1


In [278]:
df_differenced_full.fillna(0,inplace=True)   #fill na to compute first difference 

In [279]:
df_differenced_full.PM10 = first_difference(df_differenced_full.PM10) # first Difference PM10

In [280]:
df_differenced_full.CO = first_difference(df_differenced_full.CO) # first difference CO

In [281]:
df_differenced_full.NO2 =  first_difference(df_differenced_full.NO2) # first difference NO2

In [282]:
df_differenced_full.O3 =  first_difference(df_differenced_full.O3) # first difference O3

In [283]:
df_differenced_full.PM25=  first_difference(df_differenced_full.PM25) # first difference O3

In [284]:
df_differenced_full[1:5]

,date,PM10,PM10_null_pointers,CO,CO_null_pointers,NO2,NO2_null_pointers,O3,O3_null_pointers,PM25,PM25_null_pointers
1,2007-01-02,0.000000,-1,0.044527,1,0.000000,-1,0.000000,-1,0.0,-1
2,2007-01-02,0.000000,-1,0.014104,1,0.000000,-1,0.000000,-1,0.0,-1
3,2007-01-02,0.118376,1,-0.024683,1,0.013418,1,-0.081297,1,0.0,-1
4,2007-01-02,-0.118376,-1,-0.033949,-1,-0.003065,1,0.081297,-1,0.0,-1


In [286]:
import pickle as pick
with open('difference_full', 'wb') as fp:
    pick.dump(df_differenced_full, fp)

In [285]:
import pickle as pick
with open('Normalized_full', 'wb') as fp:
    pick.dump(df_Normalized, fp)

In [287]:
import pickle as pick
with open('Standardized_full', 'wb') as fp:
    pick.dump(df_Standardized, fp)

In [297]:
with open ('featureset_0', 'rb') as fp:
    itemlist = pick.load(fp)

In [306]:
itemlist[2300] # we need to add this to data sets

,,Actual,Average,Record,month,year,day,date
0,Temperature,NaN,NaN,NaN,3,2011,7,2011/3/7/
1,Mean Temperature,1 °C,-,NaN,3,2011,7,2011/3/7/
2,Max Temperature,3 °C,15 °C,- (),3,2011,7,2011/3/7/
3,Min Temperature,-1 °C,3 °C,- (),3,2011,7,2011/3/7/
4,Degree Days,NaN,NaN,NaN,3,2011,7,2011/3/7/
5,Heating Degree Days,32,NaN,NaN,3,2011,7,2011/3/7/
6,Moisture,NaN,NaN,NaN,3,2011,7,2011/3/7/
7,Dew Point,-9 °C,NaN,NaN,3,2011,7,2011/3/7/
8,Average Humidity,46,NaN,NaN,3,2011,7,2011/3/7/
9,Maximum Humidity,60,NaN,NaN,3,2011,7,2011/3/7/
